In [7]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision import transforms
from torch.utils.data import Dataset
from torchvision.models import resnet50, ResNet50_Weights
from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:

class NutritionPredictor(nn.Module):
    def __init__(self, num_outputs):
        super(NutritionPredictor, self).__init__()
        mobilenet = models.mobilenet_v2(pretrained=True)
        self.feature_extractor = mobilenet.features
        path = '~/Hack/nutrition5k_dataset_nosides/metadata/ingredients_metadata.csv'
        incr = pd.read_csv(path)
        self.cols = incr.columns
        self.tabl = np.array(incr)[:,-4:].astype(float)
        #self.feature_extractor[0][0] = nn.Conv2d(6, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.classifier = nn.Sequential(
            nn.BatchNorm1d(128),
            nn.Linear(125440, 4000),
            nn.ReLU(),
            nn.Linear(4000, 4000),
            nn.ReLU(),
            nn.Linear(4000, num_outputs)
        )
        
    def forward(self, x):
        x1 = x[:,0,:,:]
        x2 = x[:,1,:,:]
        x1 = self.feature_extractor(x1)
        x2 = self.feature_extractor(x2)
        #y2 = self.feature_extractor(x[1])
        x = torch.cat((x1.view(x1.size(0), -1), x2.view(x2.size(0), -1)), dim=-1)
        x = self.classifier(x)
        return x

# Example usage
model = NutritionPredictor(num_outputs=554)  # Define num_outputs based on your nutrition scores
for param in model.feature_extractor.parameters():
    param.requires_grad = False


/home/julius/Hack/FoodApp/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/julius/Hack/FoodApp/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:

class CustomDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.from_numpy(data).transpose(2,-1).float()
        self.targets = torch.from_numpy(targets).float()
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.targets[idx]
        if self.transform:
            x = self.transform(x)
        return x, y

In [10]:
path = '~/Hack/nutrition5k_dataset_nosides/metadata/ingredients_metadata.csv'
incr = pd.read_csv(path)
cols = incr.columns
tabl = np.array(incr)[:,-4:].astype(float)

array([[0.98, 0.043, 0.034, 0.11],
       [0.33, 0.003, 0.08, 0.007],
       [0.646, 0.034, 0.032, 0.061],
       ...,
       [2.1, 0.101, 0.04, 0.256],
       [2.02, 0.144, 0.063, 0.16],
       [8.864, 1.0, 0.0, 0.0]], dtype=object)

,ingr,id,cal/g,fat(g),carb(g),protein(g)
0,cottage cheese,1,0.980,0.043,0.034,0.110
1,strawberries,2,0.330,0.003,0.080,0.007
2,garden salad,3,0.646,0.034,0.032,0.061
3,bacon,4,5.410,0.420,0.014,0.370
4,potatoes,5,0.770,0.001,0.170,0.020
...,...,...,...,...,...,...
550,pizza sauce,551,0.290,0.002,0.070,0.013
551,pomegranate,552,0.830,0.012,0.190,0.017
552,nut cheese,553,2.100,0.101,0.040,0.256
553,soy sausage,554,2.020,0.144,0.063,0.160
